In [9]:
import torchvision
import numpy as np
import matplotlib
from src import preprocess, activation_functions, initializers, layer, loss_functions, optimizers, FFNN, training, wandblogger



In [10]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False)

In [11]:
import numpy as np
from src import preprocess

validation_percent = 0.2
one_hot_encoder = preprocess.OneHotEncoder()


train_size = int((1 - validation_percent) * len(train_dataset))
validation_size = len(train_dataset) - train_size

x_train = train_dataset.data[:train_size].reshape(train_size, -1)
targets_train = np.array(train_dataset.targets[:train_size])
y_train = one_hot_encoder.transform(targets_train)

x_valid = train_dataset.data[train_size:].reshape(validation_size, -1)
targets_valid = np.array(train_dataset.targets[train_size:])
y_valid = one_hot_encoder.transform(targets_valid)

x_test = test_dataset.data.reshape(len(test_dataset), -1)
targets_test = np.array(test_dataset.targets)
y_test = one_hot_encoder.transform(targets_test)

print(type(x_train), type(y_train))
print(x_train.shape, y_train.shape)

<class 'torch.Tensor'> <class 'numpy.ndarray'>
torch.Size([48000, 784]) (48000, 10)


/var/folders/xw/j6c1c3514_l7lyj6wyc6l1rc0000gn/T/ipykernel_11581/1812696027.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  targets_train = np.array(train_dataset.targets[:train_size])
/var/folders/xw/j6c1c3514_l7lyj6wyc6l1rc0000gn/T/ipykernel_11581/1812696027.py:16: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  targets_valid = np.array(train_dataset.targets[train_size:])
/var/folders/xw/j6c1c3514_l7lyj6wyc6l1rc0000gn/T/ipykernel_11581/1812696027.py:20: DeprecationWarn

In [12]:
#find min and max value of y_train
print("y_train min:", np.min(y_train), "y_train max:", np.max(y_train))

y_train min: 0.0 y_train max: 1.0


In [13]:
from src import (
    FFNN,
    layer,
    activation_functions,
    initializers,
    loss_functions,
    optimizers,
    batchnorm,
    dropout,
)


activation_function = activation_functions.ReLU()
identity_activation = activation_functions.Identity()

weight_init = initializers.HeInitializer()
bias_init = initializers.ConstantInitializer(0.0)

loss_fn = loss_functions.CrossEntropyLoss()
optimizer = optimizers.SGD(learning_rate=0.01)

layers = [
    layer.Layer(
        input_size= 28 * 28,
        output_size=128,
        activation_function=activation_functions.Identity(),
        weight_initializer=weight_init,
        bias_initializer=bias_init,
    ),
    batchnorm.BatchNorm(128),
    activation_functions.ReLU(),
    dropout.Dropout(0.5),
    layer.Layer(
        input_size=128,
        output_size=64,
        activation_function=activation_functions.Identity(),
        weight_initializer=weight_init,
        bias_initializer=bias_init,
    ),
    batchnorm.BatchNorm(64),
    activation_functions.ReLU(),
    dropout.Dropout(0.5),
    layer.Layer(
        input_size=64,
        output_size=10,
        activation_function=activation_functions.Identity(),
        weight_initializer=weight_init,
        bias_initializer=bias_init,
    ),
]

model = FFNN.FFNN(layers=layers, loss_function=loss_fn, optimizer=optimizer)

In [14]:
# activation_func = activation_functions.ReLU()
# loss_func = loss_functions.CrossEntropyLoss()
# optimizer = optimizers.SGD()
# W_initializers = initializers.HeInitializer()
# B_initializer = initializers.ConstantInitializer(0)
# input_size = x_train.shape[1]
# hidden_size = 392
# output_size = 10

# Input_layer = layer.Layer(input_size=input_size, output_size=output_size, activation_function=activation_func, weight_initializer=W_initializers, bias_initializer=B_initializer)
# layers = [Input_layer]
# FeedForward = FFNN.FFNN(layers=layers, loss_function=loss_func, optimizer=optimizer)


In [15]:

epochs = 100

batch_size = 32
early_stopping = training.EarlyStopping(patience=5, min_delta=1e-4)
training.Trainer(model, loss_function=loss_fn, optimizer=optimizer).train(X=x_train, y=y_train, x_val=x_valid, y_val=y_valid, early_stopper=early_stopping, epochs=epochs, batch_size=batch_size, shuffle=True)

Epoch 1/100 - Loss: 2.3982 - Val Loss: 1.9935
Epoch 2/100 - Loss: 1.8765 - Val Loss: 1.7185
Epoch 3/100 - Loss: 1.6819 - Val Loss: 1.5674
Epoch 4/100 - Loss: 1.5516 - Val Loss: 1.4593
Epoch 5/100 - Loss: 1.4571 - Val Loss: 1.3785
Epoch 6/100 - Loss: 1.3811 - Val Loss: 1.3037
Epoch 7/100 - Loss: 1.3177 - Val Loss: 1.2386
Epoch 8/100 - Loss: 1.2648 - Val Loss: 1.1806


KeyboardInterrupt: 

In [ ]:
accuracy = loss_functions.Accuracy() 
#accuracy
y_preds = model.forward_pass(x_test)
acc = accuracy.compute(y_test, y_preds)
print(f"Test accuracy: {acc * 100:.2f}%")

Test accuracy: 65.78%
